In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

### Load Amazon Data Into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

### Create DataFrames

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", 
                    "total_votes", "vine", "verified_purchase"])

# Convert star_rating column to integer type
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [6]:
# 1. Filter data and create new dataframe were total_votes is >= 20
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
votes_gt_20 = vine_df.filter(col("total_votes") >= 20)
votes_gt_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [7]:
# 2. Create dataframe that includes all rows where number of helpful_votes divided
# by total_votes is >= 0.5
helpful_gt_0p5 = votes_gt_20.filter(col("helpful_votes")/col("total_votes") >= 0.5)
helpful_gt_0p5.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [10]:
# 3. From dataframe in previous step, create new dataframe where review was written
# as part of the vine program - paid review
vine_review = helpful_gt_0p5.filter(col("vine") == "Y")
vine_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|
|R2QT2G3YRGC0PS|          5|           18|         20|   Y|                N|
| RDUP15OURNUSA|          5|           22|         23|   Y|                N|
|R1EI4NZTG7G14F|          5|           17|         23|   Y|     

In [11]:
# 4. From dataframe in previous step, create new dataframe where no review was written
# as part of the vine program - unpaid review
no_vine_review = helpful_gt_0p5.filter(col("vine") == "N")
no_vine_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [56]:
# 5. Determine the total number of reviews, 5-star reviews, and the percentage
# of 5-star reviews for the two types of review

from pyspark.sql.functions import when,count

total_paid_reviews = vine_review.count()
total_paid_5star_reviews = vine_review.filter(col("star_rating") == 5).count()
percent_paid_5star = (total_paid_5star_reviews/total_paid_reviews)*100

total_unpaid_reviews = no_vine_review.count()
total_unpaid_5star_reviews = no_vine_review.filter(col("star_rating") == 5).count()
percent_unpaid_5star = (total_unpaid_5star_reviews/total_unpaid_reviews)*100

In [57]:
# Print results to screen
print(f"Paid vs. Unpaid Review Summary:\n",
      "----------------------------\n",
      f"Total paid reviews: {total_paid_reviews}\n",
      f"Paid 5-star reviews: {total_paid_5star_reviews}\n",
      f"Percentage of paid 5-star reviews: {percent_paid_5star:.1f}\n",
      "----------------------------\n",
      f"Total unpaid reviews: {total_unpaid_reviews}\n",
      f"Unpaid 5-star reviews: {total_unpaid_5star_reviews}\n",
      f"Percentage of unpaid 5-star reviews: {percent_unpaid_5star:.1f}")

Paid vs. Unpaid Review Summary:
 ----------------------------
 Total paid reviews: 285
 Paid 5-star reviews: 163
 Percentage of paid 5-star reviews: 57.2
 ----------------------------
 Total unpaid reviews: 31545
 Unpaid 5-star reviews: 14614
 Percentage of unpaid 5-star reviews: 46.3
